In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd 

driver = webdriver.Chrome()
driver.get('https://www.redbus.in/bus-tickets/delhi-to-srinagar?fromCityId=733&toCityId=84913&fromCityName=Delhi&toCityName=Srinagar&busType=Any&onward=14-Nov-2024')
driver.maximize_window()

time.sleep(5)


view_buses_buttons = driver.find_elements(By.CSS_SELECTOR, "div.button")
if view_buses_buttons:  # This will be True if the button is found
    try:
        view_buses_buttons[0].click()  # Click the first instance of the button
        time.sleep(3)  # Wait for buses to load
        print("Clicked 'View Buses' button successfully.")
    except Exception as e:
        print("Could not click 'View Buses' button:", e)
else:
    print("'View Buses' button not found, proceeding without clicking it.")

scrolling = True
scroll_attempts = 0
previous_count = 0

while scrolling and scroll_attempts < 100:  
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)  

    bus_elements = driver.find_elements(By.CSS_SELECTOR, "div.clearfix.bus-item")
    current_count = len(bus_elements)
    print("current_count",current_count)
    
    if current_count == previous_count:
        scroll_attempts += 1  
    else:
        scroll_attempts = 0  
        previous_count = current_count 

    if scroll_attempts >= 10:
        scrolling = False

bus_list = []
for bus in bus_elements:
    try:
        bus_name = bus.find_element(By.CSS_SELECTOR, 'div.travels').text if bus.find_elements(By.CSS_SELECTOR, 'div.travels') else None
        bus_type = bus.find_element(By.CSS_SELECTOR, 'div.bus-type').text if bus.find_elements(By.CSS_SELECTOR, 'div.bus-type') else None
        departure_time = bus.find_element(By.CSS_SELECTOR, 'div.dp-time').text if bus.find_elements(By.CSS_SELECTOR, 'div.dp-time') else None
        departure_location = bus.find_element(By.CSS_SELECTOR, 'div.dp-loc').get_attribute("title") if bus.find_elements(By.CSS_SELECTOR, 'div.dp-loc') else None
        duration = bus.find_element(By.CSS_SELECTOR, 'div.dur').text if bus.find_elements(By.CSS_SELECTOR, 'div.dur') else None
        arrival_time = bus.find_element(By.CSS_SELECTOR, 'div.bp-time').text if bus.find_elements(By.CSS_SELECTOR, 'div.bp-time') else None
        arrival_location = bus.find_element(By.CSS_SELECTOR, 'div.bp-loc').get_attribute("title") if bus.find_elements(By.CSS_SELECTOR, 'div.bp-loc') else None
        fare = bus.find_element(By.CSS_SELECTOR, 'div.fare span.f-19').text if bus.find_elements(By.CSS_SELECTOR, 'div.fare span.f-19') else None
        seats_available = bus.find_element(By.CSS_SELECTOR, 'div.seat-left').text if bus.find_elements(By.CSS_SELECTOR, 'div.seat-left') else None
        rating = bus.find_element(By.CSS_SELECTOR, 'div.rating span').text if bus.find_elements(By.CSS_SELECTOR, 'div.rating span') else None

        bus_list.append({
            "Bus Name": bus_name,
            "Bus Type": bus_type,
            "Departure Time": departure_time,
            "Departure Location": departure_location,
            "Duration": duration,
            "Arrival Time": arrival_time,
            "Arrival Location": arrival_location,
            "Fare": fare,
            "Seats Available": seats_available,
            "Rating": rating
        })
    except Exception as e:
        print("Error extracting bus info:", e)

bus_df = pd.DataFrame(bus_list)
print(bus_df)

driver.quit()


Clicked 'View Buses' button successfully.
current_count 3
current_count 3
current_count 3
current_count 3
current_count 3
current_count 3
current_count 3
current_count 3
current_count 3
current_count 3
current_count 3
         Bus Name                               Bus Type Departure Time  \
0  JKRTC - 172597                  Volvo AC Seater (2+2)          13:00   
1    Zingbus Plus  Bharat Benz A/C Seater /Sleeper (2+1)          20:00   
2    Zingbus Plus                      A/C Sleeper (2+1)          21:35   

   Departure Location Duration Arrival Time        Arrival Location     Fare  \
0     Sarai Kale Khan  24h 00m        13:00               Bus Stand  2568.58   
1  Kashmere Gate ISBT  19h 20m        15:20  TRC Road (Drop by Cab)     2241   
2  Kashmere Gate ISBT  19h 30m        17:05  TRC Road (Drop by Cab)     2417   

      Seats Available Rating  
0  28 Seats available    4.5  
1  30 Seats available    5.0  
2  26 Seats available    4.4  


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
import mysql.connector

# MySQL Connection Setup
con = mysql.connector.connect(
    host="localhost",
    user="root",    
    password="Kloud@123",
    autocommit=True,
    auth_plugin='mysql_native_password'
)

mycursor = con.cursor()
mycursor.execute("USE REDBUS")

# Selenium Setup
driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/rajasthan-state-road-transport-corporation')
driver.maximize_window()
time.sleep(5)

# Loop through route details with error handling for stale elements
route_links = []

try:
    while True:
        # Fetch route elements on the current page
        route_elements = driver.find_elements(By.CSS_SELECTOR, 'a.route')
        current_page_links = [(route.text, route.get_attribute('href')) for route in route_elements]
        route_links.extend(current_page_links)

        # Locate the pagination bar
        pagination = driver.find_elements(By.CSS_SELECTOR, 'div.DC_117_paginationTable div.DC_117_pageTabs')
        
        if pagination:
            # Find the currently active page
            active_page = driver.find_element(By.CSS_SELECTOR, 'div.DC_117_pageTabs.DC_117_pageActive').text
            print(f"Current page: {active_page}")
            
            # Check for the "Next" button (in the form of a page number or arrow)
            next_page = None
            next_page_found = False
            
            # Look for the next page button by checking for the "Next" or a higher number than the active page
            for page in pagination:
                page_number = page.text
                if page_number.isdigit() and int(page_number) > int(active_page):
                    next_page = page
                    next_page_found = True
                    break

            if next_page_found:
                print("Clicking next page...")
                
                # Scroll the next page element into view
                driver.execute_script("arguments[0].scrollIntoView();", next_page)
                time.sleep(1)  # Wait for the scrolling action to complete

                # Use JavaScript to click the page button (this can bypass visibility issues)
                driver.execute_script("arguments[0].click();", next_page)
                time.sleep(5)  # Wait for the page to load
            else:
                print("No more pages in pagination.")
                break  # Exit the loop if there are no more pages
        else:
            print("No pagination detected.")
            break

except NoSuchElementException as e:
    print("Error with pagination or element not found:", e)
except Exception as e:
    print("Error handling pagination:", e)

print(f"Total routes collected: {len(route_links)}")

for route_name, route_link in route_links:
    try:
        driver.get(route_link)
        time.sleep(5)
        
        # Try to click 'View Buses' button
        view_buses_buttons = driver.find_elements(By.CSS_SELECTOR, "div.button")
        if view_buses_buttons:
            try:
                view_buses_buttons[0].click()
                time.sleep(3)
            except Exception as e:
                print(f"Could not click 'View Buses' button for route {route_name}: {e}")
                continue

        # Scrolling to load all buses
        scroll_attempts = 0
        previous_count = 0
        while scroll_attempts < 10:
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
            
            bus_elements = driver.find_elements(By.CSS_SELECTOR, "div.clearfix.bus-item")
            current_count = len(bus_elements)
            if current_count == previous_count:
                scroll_attempts += 1
            else:
                scroll_attempts = 0
                previous_count = current_count

        # Process each bus element
        for bus in bus_elements:
            try:
                bus_name = bus.find_element(By.CSS_SELECTOR, 'div.travels').text
                bus_type = bus.find_element(By.CSS_SELECTOR, 'div.bus-type').text
                departure_time = bus.find_element(By.CSS_SELECTOR, 'div.dp-time').text
                duration = bus.find_element(By.CSS_SELECTOR, 'div.dur').text
                arrival_time = bus.find_element(By.CSS_SELECTOR, 'div.bp-time').text
                fare = bus.find_element(By.CSS_SELECTOR, 'div.fare span.f-19').text
                seats_available = bus.find_element(By.CSS_SELECTOR, 'div.seat-left').text.split(' ')[0]
                rating = bus.find_element(By.CSS_SELECTOR, 'div.rating span').text

                # Check if entry exists
                check_sql = """SELECT * FROM bus_routes WHERE route_name=%s AND route_link=%s AND busname=%s AND departing_time=%s"""
                check_val = (route_name, route_link, bus_name, departure_time)
                mycursor.execute(check_sql, check_val)
                result = mycursor.fetchone()

                if not result:
                    sql = """INSERT INTO bus_routes (route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
                             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
                    val = (route_name, route_link, bus_name, bus_type, departure_time, duration, arrival_time, rating, fare, seats_available)
                    mycursor.execute(sql, val)
                    con.commit()
                else:
                    print(f"Duplicate entry found for route {route_name} and bus {bus_name}. Skipping insertion.")

            except StaleElementReferenceException:
                print("Stale element encountered while processing buses. Reloading bus elements.")
                bus_elements = driver.find_elements(By.CSS_SELECTOR, "div.clearfix.bus-item")
            except Exception as e:
                print("Error extracting bus info:", e)

    except StaleElementReferenceException:
        print(f"Stale element encountered for route {route_name}. Retrying...")
        driver.get(route_link)
        time.sleep(5)
    except Exception as e:
        print(f"Error processing route {route_name}: {e}")

driver.quit()


Current page: 1
Clicking next page...
Current page: 2
No more pages in pagination.
Total routes collected: 18
Error extracting bus info: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.rating span"}
  (Session info: chrome=125.0.6422.76); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5b53e6beee3a <unknown>
#1 0x5b53e68d845c <unknown>
#2 0x5b53e69245b5 <unknown>
#3 0x5b53e6924671 <unknown>
#4 0x5b53e6918b96 <unknown>
#5 0x5b53e69474dd <unknown>
#6 0x5b53e6918a79 <unknown>
#7 0x5b53e694767e <unknown>
#8 0x5b53e69662cc <unknown>
#9 0x5b53e6947253 <unknown>
#10 0x5b53e69171c7 <unknown>
#11 0x5b53e6917b3e <unknown>
#12 0x5b53e6bb527b <unknown>
#13 0x5b53e6bb9327 <unknown>
#14 0x5b53e6ba1dae <unknown>
#15 0x5b53e6bb9df2 <unknown>
#16 0x5b53e6b8674f <unknown>
#17 0x5b53e6bde128 <unknown>
#18 0x5b53e6bde2fb <unknown>
#19 0x5b53e6bedf6c

In [ ]:
import mysql.connector

# Establish the connection
con = mysql.connector.connect(
    host="localhost",
    user="root",    
    password="Kloud@123",
    autocommit=True,
    auth_plugin='mysql_native_password'
)

print(con)
mycursor = con.cursor()

# Execute query
mycursor.execute("SHOW DATABASES")
for x in mycursor:
    print(x)


('REDBUS',)
('SQLPYTHON1',)
('emp',)
('imdb',)
('information_schema',)
('mysql',)
('performance_schema',)
('supermarkerDB',)
('sys',)


In [24]:
mycursor.execute("USE REDBUS")

In [30]:
mycursor.execute("SHOW DATABASES")
for x in mycursor:
    print(x)

InternalError: Unread result found

In [44]:
mycursor.execute("DROP TABLE bus_routes")

In [45]:
mycursor.execute("CREATE TABLE bus_routes(id INT PRIMARY KEY AUTO_INCREMENT,route_name TEXT,route_link TEXT,busname TEXT,bustype TEXT,departing_time TIME,duration TEXT,reaching_time TIME,star_rating FLOAT,price DECIMAL,seats_available INT)")

In [29]:
mycursor.execute("show tables")
for y in mycursor:
    print(y)

InternalError: Unread result found

In [48]:
mycursor.execute("Select * from bus_routes")
for y in mycursor:
    print(y)

(1, 'Delhi to Srinagar', 'https://www.redbus.in/bus-tickets/delhi-to-srinagar', 'Zingbus Plus', 'Bharat Benz A/C Seater /Sleeper (2+1)', datetime.timedelta(seconds=72000), '19h 20m', datetime.timedelta(seconds=55200), 5.0, Decimal('2478'), 15)
(2, 'Delhi to Srinagar', 'https://www.redbus.in/bus-tickets/delhi-to-srinagar', 'Zingbus Plus', 'A/C Sleeper (2+1)', datetime.timedelta(seconds=77700), '19h 30m', datetime.timedelta(seconds=61500), 4.4, Decimal('2818'), 22)
(3, 'Jammu (j and k) to Katra (jammu and kashmir)', 'https://www.redbus.in/bus-tickets/jammu-to-katra', 'Bhawani Travels', 'A/C Seater / Sleeper (2+2)', datetime.timedelta(seconds=71400), '01h 40m', datetime.timedelta(seconds=77400), 3.8, Decimal('450'), 22)
(4, 'Jammu (j and k) to Katra (jammu and kashmir)', 'https://www.redbus.in/bus-tickets/jammu-to-katra', 'Bhawani Travels', 'A/C Seater / Sleeper (2+2)', datetime.timedelta(seconds=72000), '01h 45m', datetime.timedelta(seconds=78300), 3.8, Decimal('400'), 20)
(5, 'Katra (ja

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import wait as webdriverwait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

driver = webdriver.Chrome()
driver.get('https://www.redbus.in/bus-tickets/delhi-to-srinagar?fromCityId=733&toCityId=84913&fromCityName=Delhi&toCityName=Srinagar&busType=Any&onward=14-Nov-2024')
driver.maximize_window()

time.sleep(5)

# Check if the "View Buses" button is present and click it if it exists
view_buses_buttons = driver.find_elements(By.CSS_SELECTOR, "div.button")
if view_buses_buttons:  # This will be True if the button is found
    try:
        view_buses_buttons[0].click()  # Click the first instance of the button
        time.sleep(3)  # Wait for buses to load
        print("Clicked 'View Buses' button successfully.")
    except Exception as e:
        print("Could not click 'View Buses' button:", e)
else:
    print("'View Buses' button not found, proceeding without clicking it.")

try:
    element = webdriverwait(driver,10).until(
        EC.presence_of_all_elements_located(By.TAG_NAME,'body')
    )
finally:
    driver.quit()

Clicked 'View Buses' button successfully.


TypeError: 'module' object is not callable

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
import mysql.connector

# MySQL Connection Setup
con = mysql.connector.connect(
    host="localhost",
    user="root",    
    password="Kloud@123",
    autocommit=True,
    auth_plugin='mysql_native_password'
)

mycursor = con.cursor()
mycursor.execute("USE REDBUS")

# Selenium Setup
driver = webdriver.Chrome()
driver.get('https://www.redbus.in/travels/nbstc')
driver.maximize_window()
time.sleep(5)

route_links = []

try:
    while True:
        # Fetch route elements on the current page
        route_elements = driver.find_elements(By.CSS_SELECTOR, 'a.route')
        current_page_links = [(route.text, route.get_attribute('href')) for route in route_elements]
        route_links.extend(current_page_links)

        # Locate the pagination bar
        pagination = driver.find_elements(By.CSS_SELECTOR, 'div.DC_117_paginationTable div.DC_117_pageTabs')
        
        if pagination:
            # Find the currently active page
            active_page = driver.find_element(By.CSS_SELECTOR, 'div.DC_117_pageTabs.DC_117_pageActive').text
            print(f"Current page: {active_page}")
            
            # Check for the "Next" button (in the form of a page number or arrow)
            next_page = None
            next_page_found = False
            
            # Look for the next page button by checking for the "Next" or a higher number than the active page
            for page in pagination:
                page_number = page.text
                if page_number.isdigit() and int(page_number) > int(active_page):
                    next_page = page
                    next_page_found = True
                    break

            if next_page_found:
                print("Clicking next page...")
                
                # Scroll the next page element into view
                driver.execute_script("arguments[0].scrollIntoView();", next_page)
                time.sleep(1)  # Wait for the scrolling action to complete

                # Use JavaScript to click the page button (this can bypass visibility issues)
                driver.execute_script("arguments[0].click();", next_page)
                time.sleep(5)  # Wait for the page to load
            else:
                print("No more pages in pagination.")
                break  # Exit the loop if there are no more pages
        else:
            print("No pagination detected.")
            break

except NoSuchElementException as e:
    print("Error with pagination or element not found:", e)
except Exception as e:
    print("Error handling pagination:", e)

print(f"Total routes collected: {len(route_links)}")


Current page: 1
Clicking next page...
Current page: 2
Clicking next page...
Current page: 3
Clicking next page...
Current page: 4
Clicking next page...
Current page: 5
No more pages in pagination.
Total routes collected: 49
